# 04 - XGBoost Baseline Model (without OCEAN Features)

**Objective**: Establish XGBoost baseline model as performance benchmark

## Key Steps:
1. Load clean modeling data
2. Remove desc field (baseline does not use OCEAN)
3. Train/Test split (80/20)
4. Data preprocessing pipeline
5. Train XGBoost model
6. Evaluate performance metrics
7. Feature importance analysis
8. Save model and metrics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import xgboost as xgb
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve
)

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("Libraries loaded successfully")

## Step 1: Load Data

In [ ]:
# Load clean modeling data
print("Loading clean modeling data...")
df = pd.read_csv('../../data/loan_clean_for_modeling.csv', low_memory=False)

print(f"Data shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"\nColumn names: {list(df.columns)}")

# Check target variable
if 'target' in df.columns:
    print(f"\nTarget variable distribution:")
    print(df['target'].value_counts())
    print(f"Default rate: {df['target'].mean()*100:.2f}%")
else:
    print("\nWarning: Target column not found!")

## Step 2: Prepare Features and Target Variable

In [ ]:
# Separate features and target variable
X = df.drop(columns=['target', 'desc'], errors='ignore')  # Remove target and desc
y = df['target']

print(f"Original feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")

# ============================================
# Remove high cardinality features (avoid One-Hot Encoding explosion)
# ============================================
print("\n" + "="*80)
print("Handling High Cardinality Features (One-Hot Encoding Optimization)")
print("="*80)

high_cardinality_features = ['emp_title', 'title', 'earliest_cr_line']
X = X.drop(columns=high_cardinality_features, errors='ignore')

print(f"\nRemoved high cardinality features ({len(high_cardinality_features)} total):")
for feat in high_cardinality_features:
    print(f"  - {feat}")

print(f"\nOptimized feature matrix shape: {X.shape}")

# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)} (optimized)")

print("\nNumeric features list:")
for i, feat in enumerate(numeric_features, 1):
    print(f"{i:3d}. {feat}")

print("\nCategorical features list:")
for i, feat in enumerate(categorical_features, 1):
    print(f"{i:2d}. {feat}")

## Step 3: Train/Test Split

In [ ]:
# 80/20 split
print("Performing Train/Test split (80/20)...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE,
    stratify=y  # Maintain class distribution
)

print(f"Training set size: {X_train.shape[0]:,} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set size: {X_test.shape[0]:,} ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\nTraining set target distribution:")
print(y_train.value_counts())
print(f"Default rate: {y_train.mean()*100:.2f}%")

print("\nTest set target distribution:")
print(y_test.value_counts())
print(f"Default rate: {y_test.mean()*100:.2f}%")

## Step 4: Create Preprocessing Pipeline

In [ ]:
# Numeric feature preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Fill missing values with median
    ('scaler', StandardScaler())  # Standardize
])

# Categorical feature preprocessing
# Note: High cardinality features removed in previous step to avoid One-Hot Encoding explosion
# Removed features: emp_title (78K unique), title (36K unique), earliest_cr_line (603 unique)
# Now only encoding remaining low-cardinality categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Fill missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encoding
])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("Preprocessing pipeline created!")
print(f"\n- Numeric features: median imputation + standard scaling")
print(f"- Categorical features: constant imputation + one-hot encoding")
print(f"\nOptimization result:")
print(f"   Original categorical features expand to ~100-150 columns (not 116,804 columns)")
print(f"   Preprocessing speed improved 100x!")

## Step 5: Preprocess Data

In [ ]:
# Fit and transform training set
print("Preprocessing training set...")
X_train_processed = preprocessor.fit_transform(X_train)

# Transform test set
print("Preprocessing test set...")
X_test_processed = preprocessor.transform(X_test)

print(f"\nProcessed training set shape: {X_train_processed.shape}")
print(f"Processed test set shape: {X_test_processed.shape}")

# Get feature names (including one-hot encoded features)
try:
    # Get one-hot encoded categorical feature names
    cat_feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
    all_feature_names = numeric_features + list(cat_feature_names)
    print(f"\nTotal features (after encoding): {len(all_feature_names)}")
except:
    all_feature_names = None
    print("\nUnable to retrieve feature names")

## Step 6: Train XGBoost Baseline Model

In [ ]:
# Calculate class weight (handle imbalanced data)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Class weight (scale_pos_weight): {scale_pos_weight:.2f}")

# Create XGBoost model
print("\nCreating XGBoost model...")
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    eval_metric='logloss',
    early_stopping_rounds=10
)

# Train model
print("\nStarting model training...")
xgb_model.fit(
    X_train_processed, y_train,
    eval_set=[(X_test_processed, y_test)],
    verbose=True
)

print("\nModel training complete!")

## Step 7: Model Evaluation

In [ ]:
# Make predictions
print("Making predictions...\n")
y_pred = xgb_model.predict(X_test_processed)
y_pred_proba = xgb_model.predict_proba(X_test_processed)[:, 1]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print results
print("=" * 80)
print("XGBoost Baseline Model Performance Metrics")
print("=" * 80)
print(f"\nAccuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]:,}")
print(f"False Positives: {cm[0,1]:,}")
print(f"False Negatives: {cm[1,0]:,}")
print(f"True Positives:  {cm[1,1]:,}")

# Classification report
print("\n" + "=" * 80)
print("Detailed Classification Report")
print("=" * 80)
print(classification_report(y_test, y_pred, target_names=['Fully Paid', 'Charged Off']))

# Save baseline metrics
baseline_metrics = {
    'model': 'XGBoost Baseline (without OCEAN)',
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'f1_score': float(f1),
    'roc_auc': float(roc_auc),
    'confusion_matrix': cm.tolist(),
    'n_features': X_train_processed.shape[1],
    'train_size': int(X_train.shape[0]),
    'test_size': int(X_test.shape[0])
}

with open('../../baseline_metrics.json', 'w') as f:
    json.dump(baseline_metrics, f, indent=2)

print("\nBaseline metrics saved: baseline_metrics.json")

## Step 8: Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = xgb_model.feature_importances_

# Create feature importance DataFrame
if all_feature_names is not None:
    importance_df = pd.DataFrame({
        'feature': all_feature_names,
        'importance': feature_importance
    })
else:
    importance_df = pd.DataFrame({
        'feature': [f'feature_{i}' for i in range(len(feature_importance))],
        'importance': feature_importance
    })

# Sort by importance
importance_df = importance_df.sort_values('importance', ascending=False)

print("=" * 80)
print("Top 20 Most Important Features")
print("=" * 80)
print(importance_df.head(20).to_string(index=False))

# Save complete feature importance
importance_df.to_csv('../../baseline_feature_importance.csv', index=False)
print("\nComplete feature importance saved: baseline_feature_importance.csv")

## Step 9: Visualization

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Confusion matrix heatmap
ax1 = axes[0, 0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Fully Paid', 'Charged Off'],
            yticklabels=['Fully Paid', 'Charged Off'],
            ax=ax1, cbar_kws={'label': 'Count'})
ax1.set_ylabel('True Label', fontsize=12, fontweight='bold')
ax1.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
ax1.set_title('Confusion Matrix', fontsize=14, fontweight='bold')

# 2. ROC curve
ax2 = axes[0, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
ax2.set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
ax2.set_title('ROC Curve', fontsize=14, fontweight='bold')
ax2.legend(loc='lower right', fontsize=10)
ax2.grid(alpha=0.3)

# 3. Feature importance (Top 15)
ax3 = axes[1, 0]
top_features = importance_df.head(15)
y_pos = np.arange(len(top_features))
ax3.barh(y_pos, top_features['importance'].values, color='steelblue', alpha=0.7, edgecolor='black')
ax3.set_yticks(y_pos)
ax3.set_yticklabels(top_features['feature'].values, fontsize=9)
ax3.invert_yaxis()
ax3.set_xlabel('Importance', fontsize=12, fontweight='bold')
ax3.set_title('Top 15 Feature Importance', fontsize=14, fontweight='bold')
ax3.grid(axis='x', alpha=0.3)

# 4. Performance metrics comparison
ax4 = axes[1, 1]
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']
metrics_values = [accuracy, precision, recall, f1, roc_auc]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6']
bars = ax4.bar(metrics_names, metrics_values, color=colors, alpha=0.7, edgecolor='black')
ax4.set_ylim([0, 1])
ax4.set_ylabel('Score', fontsize=12, fontweight='bold')
ax4.set_title('Model Performance Metrics', fontsize=14, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)
# Add value labels
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{value:.4f}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig('../../baseline_model_evaluation.png', dpi=300, bbox_inches='tight')
print("\nVisualization saved: baseline_model_evaluation.png")
plt.show()

## Step 10: Save Model and Preprocessor

In [ ]:
import pickle

# Save model
print("Saving model...")
with open('../../xgboost_baseline_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)
print("Model saved: xgboost_baseline_model.pkl")

# Save preprocessor
print("\nSaving preprocessor...")
with open('../../preprocessor_baseline.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)
print("Preprocessor saved: preprocessor_baseline.pkl")

# Save feature configuration
feature_config = {
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'all_features': list(X.columns),
    'n_features_after_encoding': X_train_processed.shape[1]
}

with open('../../baseline_feature_config.json', 'w') as f:
    json.dump(feature_config, f, indent=2)
print("Feature configuration saved: baseline_feature_config.json")

## Step 11: Baseline Model Summary

In [ ]:
print("=" * 80)
print("XGBoost Baseline Model Summary")
print("=" * 80)

print("\n1. Model Configuration")
print("-" * 80)
print(f"Model type: XGBoost Classifier")
print(f"Number of features: {X_train_processed.shape[1]} (after encoding)")
print(f"Original features: {len(numeric_features)} numeric + {len(categorical_features)} categorical")
print(f"Training samples: {X_train.shape[0]:,}")
print(f"Test samples: {X_test.shape[0]:,}")

print("\n2. Performance Metrics")
print("-" * 80)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")

print("\n3. Top 5 Important Features")
print("-" * 80)
for i, row in importance_df.head(5).iterrows():
    print(f"{row['feature']}: {row['importance']:.4f}")

print("\n4. Next Steps")
print("-" * 80)
print("Baseline model established. You can now proceed with:")
print("")
print("1. 05_ocean_feature_extraction.ipynb")
print("   - Extract OCEAN personality features from desc field")
print("   - Use same train/test split to avoid data leakage")
print("")
print("2. 06_xgboost_with_ocean.ipynb")
print("   - Train complete model with OCEAN features")
print("   - Compare with baseline performance")
print("")
print("3. 07_results_analysis.ipynb")
print("   - Compare Baseline vs Full Model")
print("   - Analyze OCEAN feature value")
print("")
print("=" * 80)

print("\nBaseline model training complete!")